### The realised operational coverage estimation

As the likelihood $\tilde{p}(d_o|\boldsymbol{\theta})$ of the approximate posterior does not correspond to the generative model, its level $\alpha$ credible set $\widetilde{C}_{d_o}$ does not achieve the nominal level $\alpha$, an "operational" coverage probability is defined.

$b(d_o)= P(\boldsymbol{\theta} \in \widetilde{C}_{d_o}) = \displaystyle\int  \unicode{x1D7D9}_{\widetilde{C}_{d_o}}(\boldsymbol{\theta})p(\boldsymbol{\theta}|d_o) \text{d}\boldsymbol{\theta}$

which is not generally equal to the nominal coverage, $\alpha$.

In practice, we often generate samples from the approximate posterior $\tilde{p}(\boldsymbol{\theta}|d_o)$ using MCMC and estimate $\widetilde{C}_{d_o}$. If we denote this estimator of $\widetilde{C}_{d_o}$ by $\widehat{C}_{d_o}$, we get the realised operational coverage probability

$b_r(d_o)= P(\boldsymbol{\theta} \in \widehat{C}_{d_o})$ 

For the test signal $d_{o}$:
1. Based on the MCMC, obtain the exact posterior samples $p(\boldsymbol{\theta}|d_o)$ and approximate credible intervals $\widehat{C}_{d_o}$
2. The realised operational coverage probability can be estimated by calculating the proportion of the samples that are inside credible intervals $\widehat{C}_{d_o}$.

In [1]:
import numpy as np
from utils import FFT, freq_PSD, inner_prod, waveform
from mcmc_fun_fdot import MCMC_run

np.random.seed(2023)
#----------------------------------------------------
###
# step 1
###
#Set true parameter
#we want to estimate it using MCMC
#----------------------------------------------------

fdot_true = 1e-8

# basic settings
tmax =  120*60*60                 # Final time
fs = 2*1e-3                       # Sampling rate
delta_t = np.floor(0.01/fs)       # Sampling interval 
t = np.arange(0,tmax,delta_t)     # Form time vector from t0 = 0 to t_{n-1} = tmax. Length N [include zero]
N_t = int(2**(np.ceil(np.log2(len(t)))))   # Round length of time series to a power of two 
                                           # Length of time series

#----------------------------------------------------
###
# step 2
###
# generate signal
#----------------------------------------------------
h_true_f = FFT(waveform(fdot_true,t))  # Compute true signal in frequency domain

freq,PSD = freq_PSD(t,delta_t)  # Extract frequency bins and PSD.

data_f = h_true_f    # Construct data stream d_{o}, test signal without noise

# This is the inputs d_{j}
data_y = data_f.real # only use real parts
                    

#----------------------------------------------------
###
# step 3
###
# MCMC - parameter estimation
#----------------------------------------------------


Ntotal = 21000  # Total number of iterations
burnin = 1000   # Set burn-in. This is the amount of samples we will discard whilst looking 
            # for the true parameters

variance_noise_f = N_t * PSD / (4 * delta_t)

delta_dotf = np.sqrt(1.007508992696005e-27)

fdot_start = fdot_true - 250*delta_dotf  # Starting values

fdot_chain,lp  = MCMC_run(data_f,t,0,variance_noise_f,
                            Ntotal, burnin, fdot_start,
                            printerval = 5000, save_interval = 50, 
                            fdot_var_prop = delta_dotf**2) 

fdot_chain_est,lp_est  = MCMC_run(data_f,t,1e-6, variance_noise_f,
                            Ntotal, burnin, fdot_start,
                            printerval = 5000, save_interval = 50, 
                            fdot_var_prop = delta_dotf**2)



sample1=fdot_chain_est[burnin::10]
sample0=fdot_chain[burnin::10]

#-----------------------------------------------------------------
###
# step 4
###
# calculate the proportion of the samples that are 
# inside credible intervals made by MCMC with approximate waveform
#-----------------------------------------------------------------

# hard code is not good
a95=0
a90=0
a85=0
a80=0
n=0
for i in range(sample0.shape[0]):
    n=n+1
    if sample0[i]>=np.percentile(sample1,[2.5,97.5])[0] and sample0[i]<=np.percentile(sample1,[2.5,97.5])[1]:
        a95=a95+1
    if sample0[i]>=np.percentile(sample1,[5,95])[0] and sample0[i]<=np.percentile(sample1,[5,95])[1]:
        a90=a90+1
    if sample0[i]>=np.percentile(sample1,[7.5,92.5])[0] and sample0[i]<=np.percentile(sample1,[7.5,92.5])[1]:
        a85=a85+1
    if sample0[i]>=np.percentile(sample1,[10,90])[0] and sample0[i]<=np.percentile(sample1,[10,90])[1]:
        a80=a80+1
exact_ratio=np.array([a80,a85,a90,a95])/n

exact_ratio


array([0.611 , 0.6725, 0.7495, 0.8375])

For the test signal $d_{o}$, `exact_ratio` is the realised operational coverage for nominal level at 0.8,0.85,0.9,0.95, which is also considered as the true value of the operational coverage estimator.

In [3]:
np.save("./exact_ratio for test signal.npy",exact_ratio)
np.save("./fdot_chain_est for test signal.npy",fdot_chain_est)
np.save("./fdot_chain for test signal.npy",fdot_chain)